In [ ]:
import numpy as np
import random
import tkinter as tk
from threading import Thread
from time import sleep
from typing import List

# Particle class remains unchanged
class Particle:
    def _init_(self, position, velocity, mass, particle_type):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.mass = mass
        self.type = particle_type
        self.energy = self.calculate_kinetic_energy()

    def update_position(self, time_step):
        self.position += self.velocity * time_step

    def calculate_kinetic_energy(self):
        speed_squared = np.dot(self.velocity, self.velocity)
        return 0.5 * self.mass * speed_squared

    def detect_decay(self):
        decay_probability = 0.01
        if random.random() < decay_probability:
            if self.type == "proton":
                neutron = Particle(self.position, [0, 0, 0], self.mass - 1.67e-27, "neutron")
                positron = Particle(self.position, [0, 0, 0], 9.11e-31, "positron")
                return [neutron, positron]
        return []

# Collider class remains unchanged
class Collider:
    def _init_(self, proximity_threshold=1.0):
        self.proximity_threshold = proximity_threshold

    def detect_collision(self, p1, p2):
        distance = np.linalg.norm(p1.position - p2.position)
        return distance <= self.proximity_threshold

    def apply_momentum_conservation(self, p1, p2):
        total_momentum = p1.mass * p1.velocity + p2.mass * p2.velocity
        total_mass = p1.mass + p2.mass
        p1.velocity = total_momentum / total_mass
        p2.velocity = total_momentum / total_mass

    def fusion_event(self, p1, p2, energy_threshold):
        total_energy = p1.calculate_kinetic_energy() + p2.calculate_kinetic_energy()
        if total_energy >= energy_threshold:
            new_mass = p1.mass + p2.mass
            new_velocity = (p1.velocity * p1.mass + p2.velocity * p2.mass) / new_mass
            return Particle(p1.position, new_velocity, new_mass, "fusion_product")
        return None

    def stochastic_transform(self, p1, p2):
        new_particles = []
        if random.random() < 0.5:
            new_particles.append(Particle(p1.position, p1.velocity * 0.5, p1.mass * 0.5, "fragment1"))
        if random.random() < 0.5:
            new_particles.append(Particle(p2.position, p2.velocity * 0.5, p2.mass * 0.5, "fragment2"))
        return new_particles

# Simulation Controller with GUI
class Simulation:
    def _init_(self, num_particles=10, time_step=0.01, proximity_threshold=1.0, energy_threshold=1e-13):
        self.time_step = time_step
        self.particles = self._initialize_particles(num_particles)
        self.collider = Collider(proximity_threshold=proximity_threshold)
        self.energy_threshold = energy_threshold
        self.running = False
        self.gui_thread = None

        # GUI Setup
        self.root = tk.Tk()
        self.root.title("Particle Simulation")
        self._setup_gui()
        self.root.protocol("WM_DELETE_WINDOW", self._on_close)

    def _initialize_particles(self, num_particles) -> List[Particle]:
        particles = []
        for _ in range(num_particles):
            position = np.random.uniform(-10, 10, 3)  # Random position in 3D space
            velocity = np.random.uniform(-1, 1, 3)  # Random velocity
            mass = random.uniform(1e-27, 1e-26)  # Random mass
            particle_type = random.choice(["proton", "electron", "neutron"])
            particles.append(Particle(position, velocity, mass, particle_type))
        return particles

    def _setup_gui(self):
        tk.Label(self.root, text="Particle Count:").grid(row=0, column=0)
        self.particle_count_entry = tk.Entry(self.root)
        self.particle_count_entry.insert(0, "10")
        self.particle_count_entry.grid(row=0, column=1)

        tk.Label(self.root, text="Time Step:").grid(row=1, column=0)
        self.time_step_entry = tk.Entry(self.root)
        self.time_step_entry.insert(0, "0.01")
        self.time_step_entry.grid(row=1, column=1)

        tk.Button(self.root, text="Start", command=self.start_simulation).grid(row=2, column=0)
        tk.Button(self.root, text="Pause", command=self.pause_simulation).grid(row=2, column=1)
        tk.Button(self.root, text="Reset", command=self.reset_simulation).grid(row=2, column=2)

    def start_simulation(self):
        if not self.running:
            self.running = True
            self.gui_thread = Thread(target=self._run_simulation)
            self.gui_thread.start()

    def pause_simulation(self):
        self.running = False

    def reset_simulation(self):
        self.running = False
        num_particles = int(self.particle_count_entry.get())
        self.time_step = float(self.time_step_entry.get())
        self.particles = self._initialize_particles(num_particles)
        print("Simulation reset.")

    def _run_simulation(self):
        while self.running:
            self._simulation_step()
            sleep(self.time_step)

    def _simulation_step(self):
        for particle in self.particles:
            particle.update_position(self.time_step)
            new_particles = particle.detect_decay()
            if new_particles:
                self.particles.extend(new_particles)

        for i, p1 in enumerate(self.particles):
            for j, p2 in enumerate(self.particles[i + 1:], start=i + 1):
                if self.collider.detect_collision(p1, p2):
                    self.collider.apply_momentum_conservation(p1, p2)
                    fusion_particle = self.collider.fusion_event(p1, p2, self.energy_threshold)
                    if fusion_particle:
                        self.particles.append(fusion_particle)
                    new_particles = self.collider.stochastic_transform(p1, p2)
                    if new_particles:
                        self.particles.extend(new_particles)

    def _on_close(self):
        self.running = False
        if self.gui_thread and self.gui_thread.is_alive():
            self.gui_thread.join()
        self.root.destroy()

# Example GUI Launch
if _name_ == "_main_":
    simulation = Simulation()
    simulation.root.mainloop()